In [1]:
import numpy as np

import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
pd.set_option('expand_frame_repr', True)
import math

from os import listdir

In [2]:
files_to_parse = listdir('raw_files/monthly')
column_names = [
'GLOBALEVENTID','SQLDATE','MonthYear','Year','FractionDate','Actor1Code','Actor1Name','Actor1CountryCode','Actor1KnownGroupCode	',
'Actor1EthnicCode','Actor1Religion1Code','Actor1Religion2Code','Actor1Type1Code','Actor1Type2Code','Actor1Type3Code',
'Actor2Code','Actor2Name','Actor2CountryCode','Actor2KnownGroupCode','Actor2EthnicCode','Actor2Religion1Code',
'Actor2Religion2Code','Actor2Type1Code','Actor2Type2Code','Actor2Type3Code','IsRootEvent','EventCode','EventBaseCode',
'EventRootCode','QuadClass','GoldsteinScale','NumMentions','NumSources','NumArticles','AvgTone','Actor1Geo_Type',
'Actor1Geo_FullName','Actor1Geo_CountryCode','Actor1Geo_ADM1Code','Actor1Geo_Lat','Actor1Geo_Long','Actor1Geo_FeatureID',
'Actor2Geo_Type','Actor2Geo_FullName','Actor2Geo_CountryCode','Actor2Geo_ADM1Code','Actor2Geo_Lat','Actor2Geo_Long',
'Actor2Geo_FeatureID','ActionGeo_Type','ActionGeo_FullName','ActionGeo_CountryCode','ActionGeo_ADM1Code','ActionGeo_Lat',
'ActionGeo_Long','ActionGeo_FeatureID','DATEADDED','SOURCEURL']

columns_to_keep = ['SQLDATE', 'IsRootEvent', 'EventRootCode', 'QuadClass', 'GoldsteinScale', 'AvgTone']
#optional columns later to try with; Actor1Code, Actor2Code, Actor1Geo_ADM1Code, Actor2Geo_ADM1Code

In [3]:
#%%capture
hourly_events = pd.DataFrame({})
for file in files_to_parse:
    def daysInMonth(x):
        return {
            '201201.csv': 31,
            '201202.csv': 29,
            '201203.csv': 31,
            '201204.csv': 30,
            '201205.csv': 31,
            '201206.csv': 30,
            '201207.csv': 31,
            '201208.csv': 31,
            '201209.csv': 30,
            '201210.csv': 31,
            '201211.csv': 30,
            '201212.csv': 31
        }[x]
    df = pd.read_csv('raw_files/monthly/{file}'.format(file=file), header=None, sep="\t", names=column_names)
    df = df.set_index('GLOBALEVENTID')[columns_to_keep]
    
    splitDf = pd.DataFrame({})
    #This splits the dataframe into 24 equal parts sequentially, and marks which hour each piece belongs to
    for g, subdf in df.groupby(np.arange(len(df)) // (df.shape[0] / daysInMonth(file))):
        day = math.ceil(g)+1
        subdf['Day'] = str(str(day) if (day) > 9 else ('0' + str(day)))
        for d, subsubdf in subdf.groupby(np.arange(len(subdf)) // (subdf.shape[0] / 24)):
            hour = math.ceil(d)
            subsubdf['Hour'] = str(hour) if hour > 9 else ('0' + str(hour))
            splitDf = splitDf.append(subsubdf)
        
    df2 = splitDf
    df2['EventRootCode'] = df2['EventRootCode'].astype(pd.api.types.CategoricalDtype(categories= [i for i in range(1,21)]))
    df2['GoldsteinScale'] = df2['GoldsteinScale'] /10
    df2['AvgTone'] = df2['AvgTone'] /100
    
    
    df3 = pd.get_dummies(df2, columns=['QuadClass', 'EventRootCode', 'IsRootEvent'], prefix=['QuadClass', 'EventRootCode', 'RootEvent_TF'])    
    df3 = df3.groupby(['Day','Hour'], axis=0).mean().reset_index()

    df3['DateTime'] = str(file[:4]) + '-' + str(file[4:6]) + '-' + df3['Day'] + ' ' + df3['Hour'] + ':00'
    df4 = df3.drop(['SQLDATE', 'Hour', 'Day'], 1).set_index('DateTime')
    print(df4[['RootEvent_TF_1']].head(3))
    hourly_events = hourly_events.append(df4)
    del df

C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,19,24,36,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view

                  RootEvent_TF_1
DateTime                        
2012-01-01 00:00        0.684473
2012-01-01 01:00        0.702584
2012-01-01 02:00        0.705606


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8,9,10,11,12,13,14,19,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                  RootEvent_TF_1
DateTime                        
2012-02-01 00:00        0.675769
2012-02-01 01:00        0.635378
2012-02-01 02:00        0.670930


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8,9,10,11,12,13,14,19,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

                  RootEvent_TF_1
DateTime                        
2012-09-01 00:00        0.652253
2012-09-01 01:00        0.661021
2012-09-01 02:00        0.709484


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8,9,10,11,13,14,24,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                  RootEvent_TF_1
DateTime                        
2012-10-01 00:00        0.678874
2012-10-01 01:00        0.650082
2012-10-01 02:00        0.717879
                  RootEvent_TF_1
DateTime                        
2012-11-01 00:00        0.667784
2012-11-01 01:00        0.652577
2012-11-01 02:00        0.696907


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (8,9,11,13,14,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


                  RootEvent_TF_1
DateTime                        
2012-12-01 00:00        0.617701
2012-12-01 01:00        0.638078
2012-12-01 02:00        0.695864


In [4]:
hourly_events.shape

(8784, 28)

In [5]:
hourly_events.to_csv('2012-hourly', sep='\t')